# Metadata for new spike in data and data request ctCandi calculation

## 22.01.2025

In [1]:
import pandas as pd 
import os 
import pathlib

path_to_metadata = "./metadata"

path_to_save_output = os.path.join(path_to_metadata, "processed_metadata_20250122")
os.system(f"mkdir -p {path_to_save_output}")

old_gw_metadata = pd.read_csv(os.path.join(path_to_metadata, "SPIKE_IN_metadata", "metadata_Spike_in_silico_GW_samples_from_tissue_highdepth_15112024_bosung.csv"))
new_gw_metadata = pd.read_csv(os.path.join(path_to_metadata, "SPIKE_IN_metadata", "metadata_Spike_in_silico_GW_samples_from_tissue_highdepth_15112024.csv"))

old_tm_metadata = pd.read_csv(os.path.join(path_to_metadata, "SPIKE_IN_metadata", "metadata_Spike_in_silico_TMD_samples_from_tissue_highdepth_07102024.csv"))
new_tm_metadata = pd.read_csv(os.path.join(path_to_metadata, "SPIKE_IN_metadata", "metadata_Spike_in_silico_TMD_samples_from_tissue_highdepth_07102024_bosung.csv"))

gw_metadata = pd.concat([old_gw_metadata, new_gw_metadata], axis=0)
tm_metadata = pd.concat([old_tm_metadata, new_tm_metadata], axis=0)
tm_metadata_original = tm_metadata.copy()

hanh_metadata = pd.read_excel(os.path.join(path_to_metadata, "request_data_metadata", "metadata_comutation.xlsx"))
hanh_metadata.columns = ["SampleID", "TM_BAM"]

vi_metadata = pd.read_excel(os.path.join(path_to_metadata, "request_data_metadata", "Request_data_ctCandi_region_11.01.25.xlsx"))
vi_metadata.columns = ["SampleID", "TM_BAM"]

new_spike_in_tm_data = tm_metadata[["SampleID", "Bam_file"]]
new_spike_in_tm_data.columns = ["SampleID", "TM_BAM"]

new_spike_in_tm_data.to_csv(os.path.join(path_to_save_output, "new_spike_in_data.csv"), index=False)
vi_metadata.to_csv(os.path.join(path_to_save_output, "vi_metadata.csv"), index=False)
hanh_metadata.to_csv(os.path.join(path_to_save_output, "hanh_metadata.csv"), index=False)

tm_metadata = tm_metadata[tm_metadata["SampleID"].isin(gw_metadata["SampleID"].unique())][["SampleID", "Spike_in_label", "Spike_in_ratio"]]
tm_metadata = tm_metadata.merge(gw_metadata[["SampleID", "ichorCNA"]], right_on = "SampleID", left_on = "SampleID")
# tm_metadata.columns = ["SampleID", "Label", "spike_in_ratio", "ichorCNA"]
tm_metadata.to_csv("./metadata/SPIKE_IN_20250122.csv")

vi_metadata["group"] = "Vi_request"
hanh_metadata["group"] = "Hanh_request"

In [2]:
vi_metadata[["SampleID", "group"]].to_csv("./metadata/ctCandi_20250122_Vi.csv")
hanh_metadata[["SampleID", "group"]].to_csv("./metadata/ctCandi_20250122_Hanh.csv")

In [3]:
background_tm = tm_metadata_original[["Raw_SampleID", "Raw_bam"]]
background_tm = background_tm[background_tm["Raw_bam"].duplicated() == False]
background_tm.columns = ["SampleID", "TM_BAM"]
background_tm.to_csv("/media/hieunguyen/HNSD01/src/combine_ctcandi_ichorcna/metadata/processed_metadata_20250122/background_TM.csv", index = False)
background_gw = pd.read_csv(os.path.join("/media/hieunguyen/HNSD01/src/combine_ctcandi_ichorcna/metadata/SPIKE_IN_metadata/metadata_Spike_in_silico_background_GW_samples_15112024.csv"))
background_tm = background_tm.merge(background_gw, right_on = "Sample", left_on = "SampleID")
background_tm["Spike_in_label"] = "Control"
background_tm["Spike_in_ratio"] = 0
background_tm = background_tm[["SampleID", "Spike_in_label", "Spike_in_ratio", "ichorCNA"]]
background_tm.to_csv("/media/hieunguyen/HNSD01/src/combine_ctcandi_ichorcna/metadata/BACKGROUND.csv")